# Baseline Model

## Table of Contents
1. [Model Choice](#model-choice)
2. [Feature Selection](#feature-selection)
3. [Implementation](#implementation)
4. [Evaluation](#evaluation)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, Conv2D, MaxPooling2D, UpSampling2D, Input, concatenate

In [ ]:
# Check TF versions and CPU/GPU availability
print(tf.__version__)
print(tf.config.list_physical_devices())
print(tf.test.is_gpu_available)
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd
!ls

In [ ]:
import os
os.chdir("/content/drive/My Drive/Untrained-PINN-for-SIM-main")
print(os.getcwd())

In [ ]:
import cv2#show first picture of folder as image in notebook
img = cv2.imread('Data/LPSIM/microtubules/input_frames/1_1.png')
plt.imshow(img)

## Model Choice

[Explain why you've chosen a particular model as the baseline. This could be a simple statistical model or a basic machine learning model. Justify your choice.]


## Feature Selection

[Indicate which features from the dataset you will be using for the baseline model, and justify your selection.]


In [ ]:
# Load the dataset
# Replace 'your_dataset.csv' with the path to your actual dataset
df = pd.read_csv('your_dataset.csv')

# Feature selection
# Example: Selecting only two features for a simple baseline model
X = df[['feature1', 'feature2']]
y = df['target_variable']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Implementation

[Implement your baseline model here.]



In [1]:
# Initialize and train the baseline model
# Example for a classification problem using Logistic Regression
# model = LogisticRegression()
# model.fit(X_train, y_train)

# Your implementation code here


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Untrained-PINN-for-SIM-main')


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, Conv2D, MaxPooling2D, UpSampling2D, Input, concatenate
from skimage.metrics import peak_signal_noise_ratio, structural_similarity


In [ ]:
# Define global variables
num_im = 50      # Number of examples in dataset
frames = 24       # Number of sub-frames per example
im_dim = 480     # Image dimension (after super-res)
bg_lvl = 0       # Optional bg level subtraction
k_size = 3
learning_rate = 0.001 # Kept from the last successful training in the context.
eps = 1000 # Kept from the last successful training in the context.
BATCH_SIZE = 8 # Kept from the last successful training in the context.

# Initialize empty NumPy arrays
input_frames = np.zeros([num_im, im_dim, im_dim, frames], dtype=np.float32)
gt_frames = np.zeros([num_im, im_dim, im_dim, 1], dtype=np.float32)
lr_frames = np.zeros([num_im, im_dim, im_dim, 1], dtype=np.float32)
patterns = np.zeros([frames, im_dim, im_dim, 1], dtype=np.float32)

# Define data paths
data_name = 'microtubules'
dir_name = 'Data/LPSIM/'


In [ ]:
# Load and resize input_frames, gt_frames, and lr_frames
for i in range(1, num_im):
    for j in range(1, (frames + 1)):
        input_path = dir_name + data_name + '/input_frames/' + str(i) + '_' + str(j) + '.png'
        input_temp = cv2.imread(input_path, 0)
        if input_temp is None:
            raise FileNotFoundError(f"File not found: {input_path}")
        input_temp = cv2.resize(input_temp, dsize=(im_dim, im_dim), interpolation=cv2.INTER_CUBIC)
        input_frames[i - 1, :, :, j - 1] = input_temp

    gt_path = dir_name + data_name + '/ground_truth/' + str(i) + '.png'
    gt_temp = cv2.imread(gt_path, 0)
    if gt_temp is None:
        raise FileNotFoundError(f"File not found: {gt_path}")
    gt_temp = cv2.resize(gt_temp, dsize=(im_dim, im_dim), interpolation=cv2.INTER_CUBIC)
    gt_frames[i - 1, :, :, :] = gt_temp.reshape([im_dim, im_dim, 1]) # Reshape to add channel dimension

    lr_path = dir_name + data_name + '/low_res/' + str(i) + '.png'
    lr_temp = cv2.imread(lr_path, 0)
    if lr_temp is None:
        raise FileNotFoundError(f"File not found: {lr_path}")
    lr_temp = cv2.resize(lr_temp, dsize=(im_dim, im_dim), interpolation=cv2.INTER_CUBIC)
    lr_frames[i - 1, :, :, :] = lr_temp.reshape([im_dim, im_dim, 1]) # Reshape to add channel dimension


In [ ]:
# Load and resize patterns
for i in range(1, (frames + 1)):
    pattern_path = dir_name + data_name + '/patterns/' + str(i) + '.png'
    pattern_temp = cv2.imread(pattern_path, 0)
    if pattern_temp is None:
        raise FileNotFoundError(f"File not found: {pattern_path}")
    pattern = cv2.resize(pattern_temp, dsize=(im_dim, im_dim), interpolation=cv2.INTER_CUBIC)
    patterns[i - 1, :, :, :] = pattern.reshape([im_dim, im_dim, 1]) # Reshape to add channel dimension

# Normalize all loaded data arrays
input_frames = input_frames - bg_lvl
input_frames[input_frames < 0] = 0
if np.amax(input_frames) > 0: # Avoid division by zero
    input_frames = input_frames / np.amax(input_frames)

if np.amax(gt_frames) > 0:
    gt_frames = gt_frames / np.amax(gt_frames)

if np.amax(lr_frames) > 0:
    lr_frames = lr_frames / np.amax(lr_frames)

if np.amax(patterns) > 0:
    patterns = patterns / np.amax(patterns)

# Load and reshape PSF
psf_path = dir_name + data_name + '/psf.png'
psf = cv2.imread(psf_path, 0)
if psf is None:
    raise FileNotFoundError(f"File not found: {psf_path}")
if np.amax(psf) > 0: # Normalize PSF if max value is greater than zero
    psf = psf / np.amax(psf)
psf = np.reshape(psf, [np.ma.size(psf, 0), np.ma.size(psf, 0), 1, 1])


In [ ]:

# Define flexible U-Net architecture function
def u_net_flexible(input_channels):

    inputs = Input((im_dim, im_dim, input_channels))
    block1 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(inputs)
    block2 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(block1)
    block2 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(block2)
    down1 = MaxPooling2D(pool_size=(2,2))(block2)
    block3 = Conv2D(64, (k_size, k_size), padding="same", activation = 'relu')(down1)
    block3 = Conv2D(64, (k_size, k_size), padding="same", activation = 'relu')(block3)
    down2 = MaxPooling2D(pool_size=(2,2))(block3)
    block4 = Conv2D(128, (k_size, k_size), padding="same", activation = 'relu')(down2)
    block4 = Conv2D(128, (k_size, k_size), padding="same", activation = 'relu')(block4)
    down3 = MaxPooling2D(pool_size=(2,2))(block4)
    block5 = Conv2D(256, (k_size, k_size), padding="same", activation = 'relu')(down3)
    block5 = Conv2D(256, (k_size, k_size), padding="same", activation = 'relu')(block5)
    up1 = UpSampling2D(size=(2,2))(block5)
    cat1 = concatenate([block4,up1])
    block6 = Conv2D(128, (k_size, k_size), padding="same", activation = 'relu')(cat1)
    block6 = Conv2D(128, (k_size, k_size), padding="same", activation = 'relu')(block6)
    up2 = UpSampling2D(size=(2,2))(block6)
    cat2 = concatenate([block3,up2])
    block7 = Conv2D(64, (k_size, k_size), padding="same", activation = 'relu')(cat2)
    block7 = Conv2D(64, (k_size, k_size), padding="same", activation = 'relu')(block7)
    up3 = UpSampling2D(size=(2,2))(block7)
    cat3 = concatenate([block2,up3])
    block8 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(cat3)
    block8 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(block8)
    block9 = Conv2D(32, (k_size, k_size), padding="same", activation = 'relu')(block8)
    output = Conv2D(1, (1, 1), padding="same")(block9)
    output = tf.keras.layers.ReLU(max_value=1.0)(output)

    model = tf.keras.Model(inputs=[inputs], outputs=[output])

    return model

# Define loss and learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    learning_rate,
    decay_steps=50,
    decay_rate=0.90,
    staircase=True)

mse_loss = tf.keras.losses.MeanSquaredError()

In [ ]:
# Data preparation for batch training and augmentation
# Select a range of indices for the batch
start_idx = 0
if start_idx + BATCH_SIZE > num_im:
    start_idx = num_im - BATCH_SIZE

# Create empty lists to store augmented data
x_batch_augmented = []
y_batch_augmented = []

# Define data augmentation function
def augment_data_pair(lr_img, gt_img):
    # Random horizontal flip
    if tf.random.uniform(()) > 0.5:
        lr_img = tf.image.flip_left_right(lr_img)
        gt_img = tf.image.flip_left_right(gt_img)

    # Random vertical flip
    if tf.random.uniform(()) > 0.5:
        lr_img = tf.image.flip_up_down(lr_img)
        gt_img = tf.image.flip_up_down(gt_img)

    # Random 90-degree rotation
    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    lr_img = tf.image.rot90(lr_img, k=k)
    gt_img = tf.image.rot90(gt_img, k=k)

    return lr_img, gt_img


In [ ]:
# Loop through the selected range of images and apply augmentation
for i in range(start_idx, start_idx + BATCH_SIZE):
    # Convert NumPy arrays to TensorFlow tensors
    current_lr = tf.convert_to_tensor(lr_frames[i, :, :, :], dtype=tf.float32)
    current_gt = tf.convert_to_tensor(gt_frames[i, :, :, :], dtype=tf.float32)

    # Apply augmentation
    augmented_lr, augmented_gt = augment_data_pair(current_lr, current_gt)

    # Convert augmented TensorFlow tensors back to NumPy and append
    x_batch_augmented.append(augmented_lr.numpy())
    y_batch_augmented.append(augmented_gt.numpy())

# Convert lists to NumPy arrays
x_batch_augmented = np.array(x_batch_augmented)
y_batch_augmented = np.array(y_batch_augmented)

# Ensure final shapes
x_batch_augmented = x_batch_augmented.reshape((BATCH_SIZE, im_dim, im_dim, 1))
y_batch_augmented = y_batch_augmented.reshape((BATCH_SIZE, im_dim, im_dim, 1))


In [ ]:
# Define baseline_model function
def baseline_model():
    inputs = Input(shape=(im_dim, im_dim, 1))
    x = Conv2D(32, (k_size, k_size), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (k_size, k_size), activation='relu', padding='same')(x)
    x = Conv2D(64, (k_size, k_size), activation='relu', padding='same')(x)
    x = Conv2D(32, (k_size, k_size), activation='relu', padding='same')(x)
    outputs = Conv2D(1, (k_size, k_size), activation='relu', padding='same')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Re-instantiate the baseline model
baseline_cnn_model = baseline_model()


In [ ]:
####Fehler bei baseline model ...batch training verwechslung



# Recompile and re-train the baseline model to ensure it's ready for prediction
opt_baseline = tf.keras.optimizers.Adam(lr_schedule)
baseline_cnn_model.compile(loss=mse_loss, optimizer=opt_baseline)

baseline_model_history = baseline_cnn_model.fit(

predicted_baseline = baseline_cnn_model.predict(

# Select the first image from the batch for visualization
idx_to_visualize = 0

# Prepare predicted image
predicted_baseline_viz = predicted_baseline[idx_to_visualize, :, :, :].reshape([im_dim, im_dim])
predicted_baseline_viz = predicted_baseline_viz[20:(im_dim-20), 20:(im_dim-20)] # Crop
predicted_baseline_viz = (predicted_baseline_viz - np.amin(predicted_baseline_viz)) / (np.amax(predicted_baseline_viz) - np.amin(predicted_baseline_viz)) # Normalize

# Prepare ground truth image (from y_batch_augmented)
gt_baseline_viz = y_batch_augmented[idx_to_visualize, :, :, :].reshape([im_dim, im_dim])
gt_baseline_viz = gt_baseline_viz[20:(im_dim-20), 20:(im_dim-20)] # Crop
gt_baseline_viz = (gt_baseline_viz - np.amin(gt_baseline_viz)) / (np.amax(gt_baseline_viz) - np.amin(gt_baseline_viz)) # Normalize

# Prepare low-resolution input image (from x_batch_augmented)
lr_baseline_viz = x_batch_augmented[idx_to_visualize, :, :, :].reshape([im_dim, im_dim])
lr_baseline_viz = lr_baseline_viz[20:(im_dim-20), 20:(im_dim-20)] # Crop
lr_baseline_viz = (lr_baseline_viz - np.amin(lr_baseline_viz)) / (np.amax(lr_baseline_viz) - np.amin(lr_baseline_viz)) # Normalize

# Display images
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(gt_baseline_viz, cmap='inferno')
plt.title('Ground Truth Image')
plt.subplot(1,3,2)
plt.imshow(predicted_baseline_viz, cmap='inferno')
plt.title('Baseline CNN Result')
plt.subplot(1,3,3)
plt.imshow(lr_baseline_viz, cmap='inferno')
plt.title('Low Resolution Input')
plt.show()

# Calculate PSNR and SSIM for the baseline model

psnr_baseline = peak_signal_noise_ratio(gt_baseline_viz, predicted_baseline_viz, data_range=1.0)
ssim_baseline = structural_similarity(gt_baseline_viz, predicted_baseline_viz, data_range=1.0)

print(f"Baseline CNN Model - PSNR: {psnr_baseline:.4f}")
print(f"Baseline CNN Model - SSIM: {ssim_baseline:.4f}")

## Evaluation

[Clearly state what metrics you will use to evaluate the model's performance. These metrics will serve as a starting point for evaluating more complex models later on.]



In [ ]:
# Evaluate the baseline model
# Example for a classification problem
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)

# For a regression problem, you might use:
# mse = mean_squared_error(y_test, y_pred)

# Your evaluation code here
